# 트위터 검색어 기반 크롤링 소스코드

* 참고자료 : https://jeongwookie.github.io/2019/06/10/190610-twitter-data-crawling/

In [ ]:
import GetOldTweets3 as got
from bs4 import BeautifulSoup
import time
from random import uniform
from tqdm import tqdm_notebook
import datetime
import pandas as pd
import os
import time 
from dateutil.relativedelta import *

In [ ]:
keyword_df = pd.read_excel("키워드_최종.xlsx")
keyword_list = keyword_df['키워드'].tolist()
search_list = keyword_df['검색식'].tolist()

In [ ]:
# 가져올 범위를 정의
for i,s in enumerate(search_list): 
    tweet_df = pd.DataFrame(columns=['날짜','종류','키워드','내용'])
    
    fn = "./data/" + keyword_list[i] + ".txt"
    print(fn)
    if(os.path.exists(fn)):
        print("걸림")
        continue

    for y in range(2010,2020):
        for m in range(1,13):
            print(str(y) + "-" + str(m))
            
            days_range = []

            d = datetime.datetime.strptime(str(y) + "-" + str(m) + "-" + "01", "%Y-%m-%d")
            start=  d
            end = d + relativedelta(months=1)
            
            date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
            
            
            for date in date_generated:
                days_range.append(date.strftime("%Y-%m-%d"))

            start_date = days_range[0] 
            end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1\
            
            print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
            print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))

            
        
            while True :
                try :
                    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(s)\
                                                               .setSince(start_date)\
                                                               .setUntil(end_date)\
                                                               .setMaxTweets(-1)
                    start_time = time.time()


                    tweet = got.manager.TweetManager.getTweets(tweetCriteria,bufferLength=10000000)

                    log = "Collecting data start.. from {} to {}".format(days_range[0], days_range[-1])
                    log = log + "Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60) + "\n"
                    log = log + "=== Total num of tweets is {} ===".format(len(tweet))

                    time.sleep(uniform(30,60))         
                    break

                except :     
                    print("error")
                    time.sleep(uniform(1000,2000))     
                    print("restart")

            # 수집 with GetOldTweet3
                
            tweet_list = []
            for index in tqdm_notebook(tweet):
                content = index.text
                tweet_date = index.date.strftime("%Y-%m-%d")

                # 결과 합치기
                info_list = [tweet_date,'트위터',keyword_list[i],content]
                tweet_list.append(info_list)

                # 휴식 
                #time.sleep(uniform(0,2))
            
            temp_df = pd.DataFrame(tweet_list,columns=['날짜','종류','키워드','내용'])
            temp_df = temp_df.sort_values('날짜',ascending=False)
            tweet_df = pd.concat([tweet_df,temp_df])
            tweet_df = tweet_df.drop_duplicates(['내용'])
            tweet_df = tweet_df.reset_index(drop=True)
            
    tweet_df.to_csv(fn,index=False)
